In [ ]:
import pystan

In [ ]:
X = pd.read_csv('./variables/devision_rosenID.csv', index_col=0)
y = pd.read_csv('./variables/target.csv', index_col=0 ,names=['飲食店数'])

In [ ]:
X

In [ ]:
def zscore(x, axis = None):
    xmean = x.mean(axis=axis)
    xstd  = np.std(x, axis=axis)
    zscore = (x-xmean)/xstd
    return zscore

In [ ]:
#各変数、正規分布に従ってる？
X.iloc[:, :-1] = zscore(X.iloc[:, :-1], axis=0)

In [ ]:
rosen = dict(zip(X['路線ID'].unique(), range(1, 1+len(X['路線ID'].unique()))))
Rosen = []
for i in range(len(X)):
    Rosen.append(rosen[X['路線ID'][i]])
X = X.drop(['路線ID'], axis=1)

In [ ]:
data = dict(
    N_station=X.shape[0],
    N_explanatory=X.shape[1],
    N_rosen=len(rosen),
    X=X.values,
    Rosen=Rosen,
    Y=y['飲食店数'].values
)

In [ ]:
model = '''
data {
    int N_station;
    int N_explanatory;
    int N_rosen;
    matrix[N_station, N_explanatory] X;
    int<lower=1> Rosen[N_station];
    vector<lower=0>[N_station] Y;
}

parameters {
    real mu_a;
    real mu_b[N_explanatory];
    real<lower=0> mu_s;
    real<lower=0> s_a;
    real<lower=0> s_b[N_explanatory];
    real<lower=0> s_s;
    vector[N_rosen] a;
    matrix[N_explanatory, N_rosen] b;
    vector<lower=0>[N_rosen] s;
}

model {
    a ~ student_t(4, mu_a, s_a);
    for (i in 1:N_explanatory)
        b[i,] ~ student_t(4, mu_b[i], s_b[i]);
    s ~ student_t(4, mu_s, s_s);
    for (i in 1:N_station)
        Y[i] ~ student_t(4, a[Rosen[i]] + X[i,]*b[,Rosen[i]], s[Rosen[i]]);
}

generated quantities{
    vector[N_station] predict;
    for (i in 1:N_station)
        predict[i] = student_t_rng(4, a[Rosen[i]] + X[i,]*b[,Rosen[i]], s[Rosen[i]]);
}
'''

In [ ]:
fit = pystan.stan(model_code=model, data=data, chains=3, iter=500, warmup=100, thin=1)
#fit = pystan.stan(model_code=model, data=data, chains=4, iter=2000, warmup=500, thin=1)

In [ ]:
X.columns

In [ ]:
fit

効き具合（mu_b[:]のmeanの大きさ）は昼間人口がダントツ、続いて乗降客数、一人世帯数。それ以外は小さい。

総人口をなくすと、ほんの少し精度よくなった。

なくす前と後で、'0～14歳人口', '15～64歳人口', '65歳以上人口'の効き具合の順番が変わっている。

In [ ]:
#MCMCサンプリングの結果を抽出
ms = fit.extract(permuted=False, inc_warmup=True)
#ウォームアップ（バーンイン）のサイズを取得
iter_from = fit.sim['warmup']
#ウォームアップの区間を省く
iter_range = np.arange(iter_from, ms.shape[0])
#各変数名を取得
paraname = fit.sim['fnames_oi']

#※※※今回は全て描画したいので、こちらを使う
iter_start = np.arange(0, ms.shape[0])

In [ ]:
#seabornのcolorpalette
palette = sns.color_palette()
#おまじない？
sns.set(font_scale=1)
sns.set_style("ticks")
sns.despine(offset=10, trim=True)

#複数グラフの描画（これしか方法知らない）
fig,axes  = plt.subplots(nrows=4, ncols=3, figsize=(15,10))

for i in range(4):
    for j in range(3):
#        axes[i,j].plot(iter_start, ms[iter_start, :, i*3+j], 
#                       linewidth=3, color=palette[i*3+j])
        axes[i,j].plot(iter_start, ms[iter_start, :, i*3+j])
        axes[i,j].set_title(paraname[i*3+j])
        axes[i,j].set_xlabel('mcmc_size')
        axes[i,j].set_ylabel('parameter')
        axes[i,j].grid(True)

fig.show()

In [ ]:
summary = pd.DataFrame(data=fit.summary()['summary'], index=fit.summary()['summary_rownames'], columns=fit.summary()['summary_colnames'])

In [ ]:
summary

In [ ]:
pred = summary.query('index.str.startswith("predict")', engine='python')['50%'].values

In [ ]:
#RMSE
np.sqrt(np.mean((y['飲食店数'].values-pred)**2))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y.values, pred)

昼間人口だけのやつより少し精度悪い。
でもまあ、mu_bとか見ると変数の性質わかりやすいから、有用ではある。

In [ ]:
fig, ax = plt.subplots()
ax.scatter(pred, y['飲食店数'], edgecolors=(0, 0, 0))
ax.plot([y['飲食店数'].min(), y['飲食店数'].max()], [y['飲食店数'].min(), y['飲食店数'].max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()

In [ ]:
rosen_uniq = Rosen.unique()

value_counts().indexはvalue_counts()が同じものの順番を適当にやってるから、ダメゼッタイ

ちゃんと.unique()を使う！

In [ ]:
fig, axes = plt.subplots(nrows=22, ncols=3, figsize=(15,100))

for i in range(22):
    for j in range(3):
        plt.hold(True);
        a = summary.loc[f'a[{i*3+j}]', '50%']
        b = summary.loc[f'b[{i*3+j}]', '50%']
        xxx = stations[stations['路線ID'] == rosen_uniq[i*3+j]].sort_values(by='昼間人口')["昼間人口"]
        yyy = stations[stations['路線ID'] == rosen_uniq[i*3+j]].sort_values(by='昼間人口')["飲食店事業所数"]
        axes[i,j].plot(xxx, yyy, 'o-')
        xx = range(int(xxx.tolist()[-1]))
        yy = a + b*xx
        axes[i,j].plot(xx, yy)
        axes[i,j].set_title(f'{i*3+j}, a={a:.2f}, b={b:.4f}, {rosen_uniq[i*3+j]}')
        if i*3+j == 63:
            break
fig.show()

In [ ]:
#fig, axes = plt.subplots(nrows=7, ncols=9, figsize=(60,40))

#for i in range(7):
#    for j in range(9):
#        plt.hold(True);
#        a = summary.loc[f'a[{i*9+j}]', '50%']
#        b = summary.loc[f'b[{i*9+j}]', '50%']
#        xxx = stations[stations['路線ID'] == rosen_uniq[i*9+j]].sort_values(by='昼間人口')["昼間人口"]
#        yyy = stations[stations['路線ID'] == rosen_uniq[i*9+j]].sort_values(by='昼間人口')["飲食店事業所数"]
#        axes[i,j].plot(xxx, yyy, 'o-')
#        xx = range(int(xxx.tolist()[-1]))
#        yy = a + b*xx
#        axes[i,j].plot(xx, yy)
#        axes[i,j].set_title(f'{i*9+j}, a={a:.2f}, b={b:.4f}, {rosen_uniq[i*9+j]}')
#fig.savefig('output/hierarchical_bayes_chukan', dpi=200)
#fig.show()

In [ ]:
fig, axes = plt.subplots(nrows=8, ncols=8, figsize=(60,60))

for i in range(8):
    for j in range(8):
        plt.hold(True);
        a = summary.loc[f'a[{i*8+j}]', '50%']
        b = summary.loc[f'b[{i*8+j}]', '50%']
        s = summary.loc[f's[{i*8+j}]', '50%']
        xxx = stations[stations['路線ID'] == rosen_uniq[i*8+j]].sort_values(by='昼間人口')["昼間人口"]
        yyy = stations[stations['路線ID'] == rosen_uniq[i*8+j]].sort_values(by='昼間人口')["飲食店事業所数"]
        axes[i,j].plot(xxx, yyy, 'o-')
        xx = range(int(xxx.tolist()[-1]))
        yy = a + b*xx
        axes[i,j].plot(xx, yy)
        if i*8+j == 3:
            axes[i,j].set_title(
                f'{i*8+j}, a={a:.2f}, b={b:.4f}, s={s:.2f}, {rosen_uniq[i*8+j]}\n \
                ochiaigawa, takenami, kokokei, minosakamoto, kamado,\
                \n sakashita, mizunami, tokishi, ena, nakatugawa, tajimi')
        else:
            axes[i,j].set_title(f'{i*8+j}, a={a:.2f}, b={b:.4f}, s={s:.2f}, {rosen_uniq[i*8+j]}')
        if i*8+j == 62: #本当は64個あるけど、64+1=65個はきれいに並べれないから、最後の1個省く
            break
a = summary.loc['mu_a', '50%']
b = summary.loc['mu_b', '50%']
s = summary.loc['mu_s', '50%']
xxx = stations.sort_values(by='昼間人口')["昼間人口"]
yyy = stations.sort_values(by='昼間人口')["飲食店事業所数"]
axes[7,7].plot(xxx, yyy, 'o-')
xx = range(int(xxx.tolist()[-1]))
yy = a + b*xx
axes[7,7].plot(xx, yy)
axes[7,7].set_title(f'{0}, a={a:.2f}, b={b:.4f}, s={s:.2f}, total')

fig.savefig('output/hierarchical_bayes_chukan', dpi=200)
fig.show()

In [ ]:
a = summary.loc['a[3]', '50%']
b = summary.loc['b[3]', '50%']
xxx = stations[stations['路線ID'] == 'g_3.0'].sort_values(by='昼間人口')["昼間人口"]
yyy = stations[stations['路線ID'] == 'g_3.0'].sort_values(by='昼間人口')["飲食店事業所数"]
plt.plot(xxx, yyy, 'o-')
xx = range(int(xxx.tolist()[-1]))
yy = a + b*xx
plt.plot(xx, yy)
plt.title('a= \n \
ochiaigawa, takenami, kokokei, minosakamoto, kamado, \n sakashita, mizunami, tokishi, ena, nakatugawa, tajimi')

In [ ]:
#中央本線
#路線ID g_3.0
#summary 3

#駅数 11
#多治見、土岐、瑞浪、恵那、中津川
#5601, 2927, 2465, 3003, 4377

In [ ]:
a = summary.loc['mu_a', '50%']
b = summary.loc['mu_b', '50%']
xxx = stations.sort_values(by='昼間人口')["昼間人口"]
yyy = stations.sort_values(by='昼間人口')["飲食店事業所数"]
plt.plot(xxx, yyy, 'o-')
xx = range(int(xxx.tolist()[-1]))
yy = a + b*xx
plt.plot(xx, yy)

In [ ]:
a_summary = summary.query('index.str.contains("a")', engine='python')['50%']

In [ ]:
a_summary

In [ ]:
b_summary = summary.query('index.str.contains("b")', engine='python')['50%']

In [ ]:
b_summary

In [ ]:
s_summary = summary.query('index.str.contains("s")', engine='python')['50%']

In [ ]:
s_summary

b[3]が大きすぎて、b[4]が小さすぎる問題は直らない。なんでこうなるのか不明。　->　value_counts().indexはvalue_counts()が同じものの順番を適当にやってるから、だった。（別々の路線のプロットと回帰直線を併せて図示していた）

結論：

乗降客数で線形回帰はダメだったけど、昼間人口で線形回帰は精度も当てはまりもなかなかいい感じだった。階層ベイズなら、図示して読み取った情報を使った弱情報事前分布をa, b, sに与えないでも収束した。

瑞浪は中央本線岐阜県の駅の中では、飲食店数ふつうだった。中央本線岐阜県の駅のbは全体の2倍くらい大きいから、東海3県の駅の中では、飲食店数むしろ多かった。